In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from  sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import Model
import warnings
warnings.filterwarnings('ignore')
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pickle

[nltk_data] Downloading package stopwords to C:\Users\BASHEER-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# LOADING THE TRAIN DATASET

df= pd.read_csv("train_processed.csv",index_col = ["train_id"])
df

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,log_price,name_processed,brand_name_processed,category_name_preprocessed,Tier_1,Tier_2,Tier_3,processed_item_description
train_id,,,,,,,,,,,,,,,
0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,2.397895,mlb cincinnati reds shirt size xl,mlb,men/top/tshirts,men,top,tshirts,description yet
1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,3.970292,razer blackwidow chroma keyboard,razer,electronic/computer tablet/component parts,electronic,computer tablet,component parts,keyboard great condition works like came box p...
2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,2.397895,avaviv blouse,target,women/top blouse/blouse,women,top blouse,blouse,adorable top hint lace key hole back pale pink...
3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,3.583519,leather horse statues,missing,home/home dcor/home dcor accents,home,home dcor,home dcor accents,new tags leather horses retail stand foot high...
4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,3.806662,24k gold plated rose,missing,women/jewelry/necklaces,women,jewelry,necklaces,complete certificate authenticity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl...",3.044522,free people inspired dress,free people,women/dresse/midcalf,women,dresse,midcalf,lace says size small fits medium perfectly nev...
1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t,2.708050,little mermaid handmade dress,disney,kid/girl 2t5t/dresses,kid,girl 2t5t,dresses,little mermaid handmade dress never worn size 2t
1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape.",2.564949,21 day fix containers eating plan,missing,sport outdoor/exercise/fitnes accessories,sport outdoor,exercise,fitnes accessories,used twice still great shape


In [3]:
# SPLITTING THE DATASET
df_train,df_val = train_test_split(df,test_size=0.1,random_state = 3)

## Function 1

**THIS FUNCTION TAKES DATAFRAME AS INPUT AND RETURNS PREDICTION AS OUTPUT**

In [4]:
'''THIS FUNCTION TOKENIZES AND PADS THE FEATURE'''

def text_vectorizer(feature):
    # TOKENIZER
    tk = Tokenizer()
    # FIT ON TRAIN 
    tk.fit_on_texts(df_train[feature].apply(str))
    # TOKENIZES THE TRAIN DATASET
    tk_train = tk.texts_to_sequences(df_train[feature].apply(str))
    # TOKENIZES THE VALIDATION DATASET
    tk_val = tk.texts_to_sequences(df_val[feature].apply(str))
    
    # COMPUTES THE MAX LENGTH
    max_length = df_train[feature].apply(lambda x :len(str(x).split())).max()
    
    # COMPUTE THE VOCAB SIZE
    vocab_size = len(tk.word_index) + 1
    
    # PADDING THE TRAIN SEQUENCES
    #train_pad= pad_sequences(tk_train,padding="post",maxlen = max_length)
    # PADDING THE VALIDATION SEQUENCES
    #val_pad = pad_sequences(tk_val,padding = "post", maxlen = max_length)
    
    # RETURN THE TOKENIZER, MAX LENGTH , PADDED TRAIN SEQUENCES , PADDED VALIDATION SEQUENCES 
    return tk , max_length,vocab_size
#, train_pad , val_pad 

In [5]:
def function1(df1):

    df_raw = df1 # STORING THE INPUT DATAFRAME

  #ITEM CONDITON ID FEATURE
    item_cond = df_raw.item_condition_id # STORING THE ITEM CONDITION ID

  #######################################

  # SHIPPING FEATURE
    shipping = df_raw.shipping # STORING THE SHIPPING 

  ########################################

  # NAME FEATURE

  # Ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
    def decontracted(phrase):
        '''THIS FUNCTION DECONTRACTS THE TEXT'''
      # specific
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

      # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase
    

    st_words = stopwords.words('english') # STORING STOPWORDS 

    def name_process(text):
        '''THIS FUNCTION IS USED TO PREPROCESS THE NAME FEATURE'''
        
        text = decontracted(text) # DECONTRACTION
        text = re.sub("[^A-Za-z0-9 ]","",text) # REMOVE EVERYTHING EXCEPT THE PROVIDED CHARACTERS
        text = text.lower() # CONVERT TO LOWER CASE
        text =  " ".join([i for i in text.split() if i not in st_words])
        if len(text)==0:
            text = "missing"
        return text # RETURN THE OUTPUT TEXT
    

    df_raw["name_processed"] = df_raw.name.apply(name_process) # STORING PREPROCESSD DATA IN DATAFRAME
    df_raw[df_raw.name_processed.isnull()].name_processed ="missing"  # INPUTING NULL FOR MISSING 
    
    # TOKENIZING  
    tk_name_processed , max_length_name_processed ,vocab_size_name_processed = text_vectorizer("name_processed")
    # PADDING
    name_pad = pad_sequences(tk_name_processed.texts_to_sequences(df_raw.name_processed),maxlen=max_length_name_processed,padding="post")

##########################################################################################################################

  ## BRAND NAME FEATURE
    def brand_process(text):
        '''THIS FUNCTION PREPROCESSED THE BRAND NAME'''
        text = re.sub("[^A-Za-z0-9 ]","",text)# REMOVE EVERYTHING EXCEPT THE PROVIDED CHARACTERS
        text = text.lower()  # CONVERT TO LOWER CASE
        return text
    
    '''THIS LOOP EXTRACTS BRAND NAME FROM NAME'''
    brand_score = dict(df[df.brand_name.notnull()]["brand_name"].apply(brand_process).value_counts())

    processed_brand_name = [] #storing the barand name after preprocessing
    for index,i in df_raw.iterrows() : # for each row in the dataset
      
        if  pd.isnull(i.brand_name): #if the brand name isnull we follow this
          
            words = i.name_processed.split() # we will split the name for that datapoint
            score  = [] # this variable stores the score for each word that we calculated above
            for j in words: # for each word 
                if j in brand_score.keys(): #if the words in name is present in the keys of brand score dict
                    score.append(brand_score[j]) # take the score from the dict and append in the score variable
                else: #if the word is not a brand name append -1
                    score.append(-1)
          # once we get the scores for all the words in the name the word with maximum score woulb be the brand name
            if max(score) > 0: #if the maximum score is greater than 0 then it contains a brand name so we append the brand name
                processed_brand_name.append(words[score.index(max(score))])
            else: # if maximum value is less than 0 then it means no brand name was found so "missing" is appended
                processed_brand_name.append("missing")
              
        else: # if the brand_name is not null we follow this
            processed_brand_name.append(brand_process(i.brand_name))
    
    # STORES THE PROCESSED BRAND NAME IN DATAFRAME
    df_raw["brand_name_processed"] = processed_brand_name
    
    # TOKENIZING
    tk_brand_name,max_length_brand_name,vocab_size_brand_name = text_vectorizer("brand_name_processed")
    # PADDING
    brand_name_pad = pad_sequences(tk_brand_name.texts_to_sequences(df_raw.brand_name_processed),maxlen=max_length_brand_name,padding="post")

  ########################################################################################################################################

  ### CATEGORY NAME
    def category_name_preprocessing(text):
        ''' THIS FUNCTION PREPROCESSES THE TEXT IN "category_name" FEATURE'''
        
        text = re.sub("[^A-Za-z0-9/ ]","",text)# REMOVING ALL THE TEXT EXCEPT THE GIVEN CHARACTERS
        text = re.sub("s "," ",text) # REMOVING  "s" AT THE END OF THE WORD
        text = re.sub("s/","/",text) # REMOVING  "s" AT THE END OF THE WORD
        text = re.sub("  "," ",text) # REMOVING ONE SPACE WHERE TWO SPACES ARE PRESENT
        text = text.lower() # CONVERTING THE TEXT TO LOWER CASE
        return text # RETURNING THE PROCESSED TEXT
    
    # "missing" TO NULL VALUES
    df_raw.category_name[df_raw.category_name.isnull()] = "missing"
    # PREPROCESS CATEGORY NAME
    df_raw["category_name_preprocessed"] = df_raw.category_name.apply(category_name_preprocessing)
    
    # EXTRACT TIER 1
    df_raw["Tier_1"] = df_raw.category_name_preprocessed.apply(lambda x:   x.split("/")[0] if len(x.split("/"))>=1 else "missing")
    # TOKENIZING AND PAGGING
    tk_tier1 , max_length_tier1 ,vocab_size_tier1 = text_vectorizer("Tier_1")
    tier1_pad = pad_sequences(tk_tier1.texts_to_sequences(df_raw.Tier_1),maxlen=max_length_tier1,padding="post")
    
    # EXTRACI TIER 2
    df_raw["Tier_2"] = df_raw.category_name_preprocessed.apply(lambda x:   x.split("/")[1] if len(x.split("/"))>1 else "missing")
    # TOKENIZING AND PADDING  
    tk_tier2 , max_length_tier2 ,vocab_size_tier2 = text_vectorizer("Tier_2")
    tier2_pad = pad_sequences(tk_tier2.texts_to_sequences(df_raw.Tier_2),maxlen=max_length_tier2,padding="post")
    
    # EXTRACT TIER3
    df_raw["Tier_3"] = df_raw.category_name_preprocessed.apply(lambda x:   x.split("/")[2] if len(x.split("/"))>1 else "missing")
    #TOKENIZING AND PADDING
    tk_tier3 , max_length_tier3 ,vocab_size_tier3 = text_vectorizer("Tier_3")
    tier3_pad = pad_sequences(tk_tier3.texts_to_sequences(df_raw.Tier_3),maxlen=max_length_tier3,padding="post")
  #########################################################################################################################################

  # ITEM DESCRIPTION 
    
    def processing_item_description(text):
        '''THIS FUNCTION PREPROCESSES THE TEXT IN "item_description"'''
        text = re.sub("\[rm\] ","",str(text))
        text = decontracted(text)
        text = re.sub("[^A-Za-z0-9 ]","",str(text))
        text = str(text).lower()
        text =  " ".join([i for i in text.split() if i not in st_words])
        if len(text)==0:
            text = "missing"
        return text
    # "missing" TO NULL VALUES
    df_raw.item_description[df_raw.item_description.isnull()]="missing"
    df_raw["processed_item_description"] = df_raw.item_description.apply(processing_item_description)
    # TOKENIZING AND PADDING
    tk_desc , max_len_desc ,vocab_size_desc = text_vectorizer("processed_item_description")
    desc_pad = pad_sequences(tk_desc.texts_to_sequences(df_raw.processed_item_description),maxlen=max_len_desc,padding="post")
    
    # FORMING INPUT TO MODEL BY FORMING LIST OF ALL FEATURES
    x = [item_cond,shipping,brand_name_pad,tier1_pad,tier2_pad,tier3_pad,name_pad,desc_pad]

  ########################################################################################################################################

  # MODEL ARCHITECTURE

  # ITEM CONDITION ID
    inp1 = layers.Input(shape=(1))
    emb1  = layers.Embedding(6,10,input_length=1)(inp1)
    flat1 = layers.Flatten()(emb1)

  # SHIPPING 
    inp2 = layers.Input(shape=(1))
    d2 = layers.Dense(10,activation="relu")(inp2)

  # BRAND NAME
    inp3 = layers.Input(shape= (8))
    emb3 = layers.Embedding(vocab_size_brand_name ,16 ,input_length= 8 )(inp3)
    flat3 = layers.Flatten()(emb3)

  # Tier_1
    inp4 = layers.Input(shape = (2))
    emb4 = layers.Embedding(vocab_size_tier1, 16 , input_length=2 )(inp4)
    flat4 = layers.Flatten()(emb4)

  # Tier_2
    inp5= layers.Input(shape = (4))
    emb5 = layers.Embedding(vocab_size_tier2 , 16 ,input_length= 4 )(inp5)
    flat5 = layers.Flatten()(emb5)

  # Tier_3
    inp6= layers.Input(shape = (6))
    emb6 = layers.Embedding(vocab_size_tier3, 16 ,input_length= 6 )(inp6)
    flat6 = layers.Flatten()(emb6)

  # NAME PROCESSED
    inp7= layers.Input(shape = (13))
    emb7 = layers.Embedding(vocab_size_name_processed,20 ,input_length= 13 )(inp7)
    lstm7 = layers.GRU(64,return_sequences=True)(emb7)
    flat7 = layers.Flatten()(lstm7)

  # ITEM DESCRIPTION
    inp8= layers.Input(shape = (193))
    emb8 = layers.Embedding(vocab_size_desc , 40 , input_length= 193 )(inp8)
    lstm8 = layers.GRU(64,return_sequences=True)(emb8)
    flat8 = layers.Flatten()(lstm8)

    concat = layers.Concatenate()([flat1,d2,flat3,flat4,flat5,flat6,flat7,flat8])

    dense1 = layers.Dense(512,activation="relu")(concat)
    drop2 = layers.Dropout(0.2)(dense1)

    dense2 = layers.Dense(256,activation="relu")(drop2)
    drop2 = layers.Dropout(0.3)(dense2)


    dense3 = layers.Dense(128,activation="relu")(drop2)
    drop2 = layers.Dropout(0.4)(dense3)
    bn2  = layers.BatchNormalization()(drop2)

    dense4 = layers.Dense(1,activation="linear")(bn2)

    model =  Model(inputs= [inp1,inp2,inp3,inp4,inp5,inp6,inp7,inp8],outputs=dense4)
    # COMPILING MODEL
    model.compile(optimizer="adam",loss="mse",metrics=  tf.keras.metrics.RootMeanSquaredError())
    # LOADING THE WEIGHTS
    model.load_weights("best.h5")
    
    # PREDICT THE OUTPUT
    x_pred = model.predict(x,batch_size=100)
    
    # LOG PRICE TO ACTUAL VALUES
    def log_to_actual(log):
        return np.exp(log)-1
    
    # FORING DATAFRAME OF RESULTS
    x_pred_df = pd.DataFrame(log_to_actual(x_pred),columns=["price"])

    return x_pred_df # RETURN PREDICTIONS


In [6]:
# RESULTS OF  FIRST 10 ROWS OF DATAFRAME
function1(df[:10])

1/1 [==============================] - 1s 740ms/step


,price
0,8.488786
1,48.758106
2,12.984865
3,22.767696
4,29.523535
5,40.599293
6,51.837479
7,18.560535
8,14.544349
9,21.807371


In [7]:
# PREDICTION OF ONE DATAPOINT

function1(df_val[:1])

1/1 [==============================] - 1s 595ms/step


,price
0,8.446921


In [14]:
function1(df_val[:2])

1/1 [==============================] - 1s 673ms/step


,price
0,8.446923
1,13.492451


In [15]:
function1(df_val[:5])

1/1 [==============================] - 1s 573ms/step


,price
0,8.446921
1,13.492451
2,25.166567
3,18.026264
4,16.765070


In [9]:
# PREDICTION OF VALIDATION DATA
function1(df_val)

1483/1483 [==============================] - 122s 82ms/step


,price
0,8.446923
1,13.492451
2,25.166567
3,18.026264
4,16.765070
...,...
148249,11.129321
148250,10.506555
148251,13.059233
148252,22.025299


## Function 2

**THIS FUNCTION TAKES DATAFRAME AS INPUT AND RETURNS METRIC RMSLE AS OUTPUT**

In [10]:
def function2(df1,y_price):

    df_raw = df1

  #ITEM CONDITON ID FEATURE
    item_cond = df_raw.item_condition_id

  ################################################.

  # SHIPPING FEATURE
    shipping = df_raw.shipping

  #######################################################

  # NAME FEATURE

  # Ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
    def decontracted(phrase):
      # specific
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase

    st_words = stopwords.words('english')

    def name_process(text):
        '''THIS FUNCTION IS USED TO PREPROCESS THE NAME FEATURE'''
        text = decontracted(text)
        text = re.sub("[^A-Za-z0-9 ]","",text) # REMOVE EVERYTHING EXCEPT THE PROVIDED CHARACTERS
        text = text.lower() # CONVERT TO LOWER CASE
        text =  " ".join([i for i in text.split() if i not in st_words])
        if len(text)==0:
            text = "missing"
        return text # RETURN THE OUTPUT TEXT

    df_raw["name_processed"] = df_raw.name.apply(name_process)
    df_raw[df_raw.name_processed.isnull()].name_processed ="missing" 

    tk_name_processed , max_length_name_processed ,vocab_size_name_processed = text_vectorizer("name_processed")
    name_pad = pad_sequences(tk_name_processed.texts_to_sequences(df_raw.name_processed),maxlen=max_length_name_processed,padding="post")

  ############################################################################################################################################

  ## BRAND NAME FEATURE
    def brand_process(text):
        text = re.sub("[^A-Za-z0-9 ]","",text)# REMOVE EVERYTHING EXCEPT THE PROVIDED CHARACTERS
        text = text.lower()  # CONVERT TO LOWER CASE
        return text


    brand_score = dict(df[df.brand_name.notnull()]["brand_name"].apply(brand_process).value_counts())

    processed_brand_name = [] #storing the barand name after preprocessing
    for index,i in df_raw.iterrows() : # for each row in the dataset
      
        if  pd.isnull(i.brand_name): #if the brand name isnull we follow this
          
            words = i.name_processed.split() # we will split the name for that datapoint
            score  = [] # this variable stores the score for each word that we calculated above
            for j in words: # for each word 
                if j in brand_score.keys(): #if the words in name is present in the keys of brand score dict
                    score.append(brand_score[j]) # take the score from the dict and append in the score variable
                else: #if the word is not a brand name append -1
                    score.append(-1)
          # once we get the scores for all the words in the name the word with maximum score woulb be the brand name
            if max(score) > 0: #if the maximum score is greater than 0 then it contains a brand name so we append the brand name
                processed_brand_name.append(words[score.index(max(score))])
            else: # if maximum value is less than 0 then it means no brand name was found so "missing" is appended
                processed_brand_name.append("missing")
              
        else: # if the brand_name is not null we follow this
            processed_brand_name.append(brand_process(i.brand_name))

    df_raw["brand_name_processed"] = processed_brand_name

    tk_brand_name,max_length_brand_name,vocab_size_brand_name = text_vectorizer("brand_name_processed")
    brand_name_pad = pad_sequences(tk_brand_name.texts_to_sequences(df_raw.brand_name_processed),maxlen=max_length_brand_name,padding="post")

  #############################################################################################################################################

  ### CATEGORY NAME

    def category_name_preprocessing(text):
        ''' THIS FUNCTION PREPROCESSES THE TEXT IN "category_name" FEATURE'''
        text = re.sub("[^A-Za-z0-9/ ]","",text)# REMOVING ALL THE TEXT EXCEPT THE GIVEN CHARACTERS
        text = re.sub("s "," ",text) # REMOVING  "s" AT THE END OF THE WORD
        text = re.sub("s/","/",text) # REMOVING  "s" AT THE END OF THE WORD
        text = re.sub("  "," ",text) # REMOVING ONE SPACE WHERE TWO SPACES ARE PRESENT
        text = text.lower() # CONVERTING THE TEXT TO LOWER CASE
        return text # RETURNING THE PROCESSED TEXT
    
    df_raw.category_name[df_raw.category_name.isnull()] = "missing"
    df_raw["category_name_preprocessed"] = df_raw.category_name.apply(category_name_preprocessing)

    df_raw["Tier_1"] = df_raw.category_name_preprocessed.apply(lambda x:   x.split("/")[0] if len(x.split("/"))>=1 else "missing")

    tk_tier1 , max_length_tier1 ,vocab_size_tier1 = text_vectorizer("Tier_1")
    tier1_pad = pad_sequences(tk_tier1.texts_to_sequences(df_raw.Tier_1),maxlen=max_length_tier1,padding="post")


    df_raw["Tier_2"] = df_raw.category_name_preprocessed.apply(lambda x:   x.split("/")[1] if len(x.split("/"))>1 else "missing")

    tk_tier2 , max_length_tier2 ,vocab_size_tier2 = text_vectorizer("Tier_2")
    tier2_pad = pad_sequences(tk_tier2.texts_to_sequences(df_raw.Tier_2),maxlen=max_length_tier2,padding="post")

    df_raw["Tier_3"] = df_raw.category_name_preprocessed.apply(lambda x:   x.split("/")[2] if len(x.split("/"))>1 else "missing")

    tk_tier3 , max_length_tier3 ,vocab_size_tier3 = text_vectorizer("Tier_3")
    tier3_pad = pad_sequences(tk_tier3.texts_to_sequences(df_raw.Tier_3),maxlen=max_length_tier3,padding="post")

  ##################################################################################################################################

  # ITEM DESCRIPTION 
    
    def processing_item_description(text):
        '''THIS FUNCTION PREPROCESSES THE TEXT IN "item_description"'''
        text = re.sub("\[rm\] ","",str(text))
        text = decontracted(text)
        text = re.sub("[^A-Za-z0-9 ]","",str(text))
        text = str(text).lower()
        text =  " ".join([i for i in text.split() if i not in st_words])
        if len(text)==0:
            text = "missing"
        return text

    df_raw.item_description[df_raw.item_description.isnull()]="missing"
    df_raw["processed_item_description"] = df_raw.item_description.apply(processing_item_description)

    tk_desc , max_len_desc ,vocab_size_desc = text_vectorizer("processed_item_description")
    desc_pad = pad_sequences(tk_desc.texts_to_sequences(df_raw.processed_item_description),maxlen=max_len_desc,padding="post")

    x = [item_cond,shipping,brand_name_pad,tier1_pad,tier2_pad,tier3_pad,name_pad,desc_pad]

  ##################################################################################################################################

  # ITEM CONDITION ID
    inp1 = layers.Input(shape=(1))
    emb1  = layers.Embedding(6,10,input_length=1)(inp1)
    flat1 = layers.Flatten()(emb1)


  # SHIPPING 
    inp2 = layers.Input(shape=(1))
    d2 = layers.Dense(10,activation="relu")(inp2)


    # BRAND NAME
    inp3 = layers.Input(shape= (8))
    emb3 = layers.Embedding(vocab_size_brand_name ,16 ,input_length= 8 )(inp3)
    flat3 = layers.Flatten()(emb3)

    # Tier_1
    inp4 = layers.Input(shape = (2))
    emb4 = layers.Embedding(vocab_size_tier1, 16 , input_length=2 )(inp4)
    flat4 = layers.Flatten()(emb4)

    # Tier_2
    inp5= layers.Input(shape = (4))
    emb5 = layers.Embedding(vocab_size_tier2 , 16 ,input_length= 4 )(inp5)
    flat5 = layers.Flatten()(emb5)

    # Tier_3
    inp6= layers.Input(shape = (6))
    emb6 = layers.Embedding(vocab_size_tier3, 16 ,input_length= 6 )(inp6)
    flat6 = layers.Flatten()(emb6)

    # NAME PROCESSED
    inp7= layers.Input(shape = (13))
    emb7 = layers.Embedding(vocab_size_name_processed,20 ,input_length= 13 )(inp7)
    lstm7 = layers.GRU(64,return_sequences=True)(emb7)
    flat7 = layers.Flatten()(lstm7)

    # ITEM DESCRIPTION
    inp8= layers.Input(shape = (193))
    emb8 = layers.Embedding(vocab_size_desc , 40 , input_length= 193 )(inp8)
    lstm8 = layers.GRU(64,return_sequences=True)(emb8)
    flat8 = layers.Flatten()(lstm8)

    concat = layers.Concatenate()([flat1,d2,flat3,flat4,flat5,flat6,flat7,flat8])

    dense1 = layers.Dense(512,activation="relu")(concat)
    drop2 = layers.Dropout(0.2)(dense1)

    dense2 = layers.Dense(256,activation="relu")(drop2)
    drop2 = layers.Dropout(0.3)(dense2)


    dense3 = layers.Dense(128,activation="relu")(drop2)
    drop2 = layers.Dropout(0.4)(dense3)
    bn2  = layers.BatchNormalization()(drop2)

    dense4 = layers.Dense(1,activation="linear")(bn2)
    # MODEL
    model =  Model(inputs= [inp1,inp2,inp3,inp4,inp5,inp6,inp7,inp8],outputs=dense4)
    model.compile(optimizer="adam",loss="mse",metrics=  tf.keras.metrics.RootMeanSquaredError())
    #LOAD WEIGHTS
    model.load_weights("best.h5")
    
    # EVALUATE THE RMSLE VALUES
    rmse = model.evaluate(x,np.log(y_price+1),verbose=1,batch_size=1000)
  
    return rmse[1] # RETURN RMSLE

In [11]:
# EVALUATION OF METRIC FOR VALIDATION DATA
function2(df_val,df_val.price)

149/149 [==============================] - 42s 273ms/step - loss: 0.1853 - root_mean_squared_error: 0.4304


0.4304217994213104